<a href="https://colab.research.google.com/github/udbhav-chugh/TextCoherenceOnSocialMedia/blob/master/Text_Coherence_Dataset_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook contains code to synthesize the two datasets whose details are mentioned in the report:


1.   Where the Reddit posts and comments are considered separately
2.   Where the Reddit Posts and top 10 comments are considered.

Note to run this notebook successfully, make sure to have the redditDataset folder in your google drive root directory.

Use the following link to download the zip dataset folder
https://www.kaggle.com/ammar111/reddit-top-1000

Then extract it, rename it to redditDataset and upload it on your drive before running the steps ahead.





In [ ]:
!pip install kora

import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
from kora.selenium import wd
from selenium.webdriver import ActionChains
from nltk.tokenize import sent_tokenize
import random
from google.colab import files
from google.colab import drive
import nltk
nltk.download('punkt')

     |████████████████████████████████| 61kB 3.3MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#get file names from the redditDataset folder
mypath = '/content/gdrive/My Drive/redditDataset'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)

['gifs.csv', 'AskReddit.csv', 'Jokes.csv', 'Showerthoughts.csv', 'pics.csv', 'food.csv', 'worldnews.csv', 'woahdude.csv', 'aww.csv', 'funny.csv', 'books.csv', 'IAmA.csv', 'videos.csv', 'LifeProTips.csv', 'todayilearned.csv', 'GetMotivated.csv', 'movies.csv', 'explainlikeimfive.csv']


In [ ]:
#function to get urls listed in the csv file passed as parameter
def getUrls(file):
  urls = []
  df = pd.read_csv(join(mypath, file))
  # cols = df.columns.tolist()
  urlCol = df['permalink']
  # print(len(urlCol))
  for key, value in urlCol.iteritems():
    urls.append(value)
  return urls

# Type 1 Dataset: Posts and Comments considered separately

In [ ]:
fileToWrite = '/content/gdrive/My Drive/redditPostsAndCommentsSeparate.txt'
with open(fileToWrite, 'w') as f:
  f.write('')

for file in ['AskReddit.csv','explainlikeimfive.csv','books.csv', 'Jokes.csv']:
  urls = getUrls(file)
  count=0
  for url in urls:
    if count >= 70:
      break
    wd.get(url)
    wd.implicitly_wait(1000)
    elem = wd.find_elements_by_xpath("//div[@data-click-id='text']")
    if(len(elem)==0):
      continue

    post=elem[0]
    if(len(post.text) >= 1000):
      count+=1
      with open(fileToWrite, 'a') as f:
        f.write(post.text.replace("\n", " ")+'\n')

    comments = wd.find_elements_by_xpath("//div[@data-test-id='comment']")
    for comment in comments:
      if(len(comment.text) >= 1000):
        count+=1
        with open(fileToWrite, 'a') as f:
          f.write(comment.text.replace("\n", " ")+'\n')
  print("Data extracted from ",file)

print("Separate Post and Comment Data extracted")

Data extracted from  AskReddit.csv
Data extracted from  explainlikeimfive.csv
Data extracted from  books.csv
Data extracted from  Jokes.csv
Separate Post and Comment Data extracted


# Type 2 Dataset: Posts and Top 10 Comments considered Together

In [ ]:
fileToWrite = '/content/gdrive/My Drive/redditPostsAndCommentsCombined.txt'

with open(fileToWrite, 'w') as f:
  f.write('')

for file in ['AskReddit.csv','explainlikeimfive.csv','books.csv', 'Jokes.csv']:
  urls = getUrls(file)
  count=0
  for url in urls:
    if count >= 70:
      break
    wd.get(url)
    wd.implicitly_wait(1000)
    elem = wd.find_elements_by_xpath("//div[@data-click-id='text']")
    if(len(elem)==0):
      continue

    post=elem[0]
    finalText = post.text
    comments = wd.find_elements_by_xpath("//div[@data-test-id='comment']")
    commentCount = 0
    for comment in comments:
      commentCount +=1
      if(commentCount>10):
        break
      finalText = finalText + comment.text

    if(len(finalText) >= 1000):
      count+=1
      with open(fileToWrite, 'a') as f:
        f.write(finalText.replace("\n", " ")+'\n')

  print("Data extracted from ",file)

print("Combined Post and Comment Data extracted")

Data extracted from  AskReddit.csv
Data extracted from  explainlikeimfive.csv
Data extracted from  books.csv
Data extracted from  Jokes.csv
Combined Post and Comment Data extracted


In [ ]:
#Function to receive a text input, return list of permutations of the original text which can later be paired up with original text while tra 
def getPermutations(text):
  tokenizedText = sent_tokenize(text)
  permutedTexts=[]
  for i in range(20):
    random.shuffle(tokenizedText)
    result = ''
    for sentence in tokenizedText:
      result += sentence
      result += ' '
    permutedTexts.append(result)
  
  return permutedTexts

In [ ]:
def getDataset(redditPostsAndComments, finalDatasetFile):
  with open(redditPostsAndComments, 'r') as f:
    redditPosts = f.readlines()

  with open(finalDatasetFile, 'w') as f:
    f.write('')

  for post in redditPosts:
    post = post.replace("\t", " ")
    post = post.replace("\n", " ")
    permutedPosts = getPermutations(post)
    for permutedPost in permutedPosts:
      dataPair = post + "\t" + permutedPost
      with open(finalDatasetFile, 'a') as f:
        f.write(dataPair+"\n")
  print("Dataset file generated successfully.")

In [ ]:
#get type 1 dataset
getDataset('/content/gdrive/My Drive/redditPostsAndCommentsSeparate.txt', '/content/gdrive/My Drive/datasetPostsAndCommentsSeparate.tsv')

#get type 2 dataset
getDataset('/content/gdrive/My Drive/redditPostsAndCommentsCombined.txt', '/content/gdrive/My Drive/datasetPostsAndCommentsCombined.tsv')

Dataset file generated successfully.
Dataset file generated successfully.


In [ ]:
#get training accident dataset
getDataset('/content/gdrive/My Drive/trainingAccidents.txt', '/content/gdrive/My Drive/datasetTrainingAccidents.tsv')

#get testing accident dataset
getDataset('/content/gdrive/My Drive/testingAccidents.txt', '/content/gdrive/My Drive/datasetTestingAccidents.tsv')

Dataset file generated successfully.
Dataset file generated successfully.
